In [2]:
from keplergl import KeplerGl
import geopandas as gpd
import networkx as nx
import os
from shapely.geometry import Point
import pandas as pd
import pyproj
from shapely.ops import unary_union
import hashlib
import numpy as np

# Load your data
neighborhoods = gpd.read_file(r"D:\IAAC\Thesis\Python\MLloading\Geojson\GNN_Read_data\neighborhoods_with_ndvi_numerical.geojson")
buildings = gpd.read_file(r"D:\IAAC\Thesis\Python\MLloading\Geojson\GNN_Read_data\Taipei_Buildings_fulldata.geojson")
roads = gpd.read_parquet(r"D:\IAAC\Thesis\Python\MLloading\Geojson\GNN_Read_data\taipei_segments_cleaned_verified.geoparquet")
trees = gpd.read_parquet(r"D:\IAAC\Thesis\Python\MLloading\Geojson\GNN_Read_data\taipei_land.geoparquet")
transit = gpd.read_parquet(r"D:\IAAC\Thesis\Python\MLloading\Geojson\GNN_Read_data\taipei_infrastructure.geoparquet")

# Filter transit data
transit = transit[transit['class'].isin(['stop_position', 'bus_stop'])]

# Clean transit data
# Replace NaN with None (JSON null)
transit = transit.replace({np.nan: None})

# Convert object columns to strings or handle arrays
for column in transit.select_dtypes(include=['object']).columns:
    if column == 'geometry':
        continue  # Skip the geometry column
    if transit[column].apply(lambda x: isinstance(x, (list, dict, np.ndarray))).any():
        # Convert arrays or dictionaries to strings
        transit[column] = transit[column].apply(lambda x: str(x) if x is not None else None)
    else:
        # Ensure all values are strings
        transit[column] = transit[column].astype(str).replace('nan', None)

# Convert CRS
neighborhoods = neighborhoods.to_crs('EPSG:4326')
buildings = buildings.to_crs('EPSG:4326')
roads = roads.to_crs('EPSG:4326')
trees = trees[trees['subtype'] == 'tree'].to_crs('EPSG:4326')
transit = transit.to_crs('EPSG:4326')

# Create a sample graph
G = nx.Graph()
G.add_node("sample_node", type='neighborhood', geometry=neighborhoods.iloc[0].geometry)

# Initialize Kepler.gl map
map_1 = KeplerGl(height=600, width=800, mapbox_api_access_token="pk.eyJ1Ijoiam9obm55Ym95NjY5NyIsImEiOiJjbTgyMzZkbjUxZHF2MmlzYTByc3pxZmw0In0.7RASB6M_AczC7q8dvFPWBQ")
map_1.add_data(data=neighborhoods, name="Neighborhoods")
map_1.add_data(data=buildings, name="Buildings")
map_1.add_data(data=roads, name="Roads")
map_1.add_data(data=trees, name="Trees")
map_1.add_data(data=transit, name="Transit")

# Display the map in Jupyter
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


C:\Users\陳正恆\AppData\Roaming\Python\Python312\site-packages\jupyter_client\session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)
C:\Users\陳正恆\AppData\Roaming\Python\Python312\site-packages\jupyter_client\session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


KeplerGl(data={'Neighborhoods': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 1…

In [13]:
map_1.config

{'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': '70cx7hh',
     'type': 'geojson',
     'config': {'dataId': 'Neighborhoods',
      'label': 'Neighborhoods',
      'color': [136, 87, 44],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': 'geometry'},
      'isVisible': False,
      'visConfig': {'opacity': 0.8,
       'strokeOpacity': 1,
       'thickness': 0.5,
       'strokeColor': [255, 153, 31],
       'colorRange': {'name': 'Uber Viz Sequential 4',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#E6FAFA',
         '#C1E5E6',
         '#9DD0D4',
         '#75BBC1',
         '#4BA7AF',
         '#00939C']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
   

In [14]:
import json
# After customizing the map, save the configuration to a .json file
with open(r"D:\IAAC\Thesis\Python\MLloading\Geojson\GNN_Read_data\kepler.gl.json", 'w') as f:
    json.dump(map_1.config, f, indent=2)
print("Configuration saved to kepler.gl.json")

Configuration saved to kepler.gl.json
